In [35]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output, State, ALL, MATCH
from dash import callback_context as ctx
import dash_bootstrap_components as dbc
import plotly.express as px
import json

df = pd.read_csv('data/train.csv')

columns = list(df.columns)
numeric_column = ['bant_submit', 'historical_existing_cnt', 'lead_desc_length']
target_column = [columns[-1]]
categorical_column = [col for col in columns[:-1] if col not in numeric_column]
column_dict = {'target': target_column, 'categorical': categorical_column, 'numeric': numeric_column}

In [36]:
def generate_table(column_type):
    choosed_columns = column_dict[column_type]
    column_contents = ['Column_Name', 'Description', 'Dtype', 'Null_Percentage']

    table = html.Table([
                        html.Thead(
                            html.Tr([html.Th(col) for col in column_contents])
                        ),
                        html.Tbody([
                            html.Tr([html.Td(s) for s in [
                                dbc.Button(col, id={"type": "button", "id": col}), col, str(df[col].dtype), round(df.isna().sum()[col]/len(df), 2)
                            ]]) for col in choosed_columns])
                        ])
    
    return table

In [37]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

column_types = ['target', 'categorical', 'numeric']

tabs = html.Div([
    dbc.Tabs(
        [dbc.Tab(label=col_type.title(), tab_id=col_type) for col_type in column_types],
        id='tabs',
        active_tab=column_types[0],
    ),
    html.Div([html.Table(id='content')]),
], style={'width': '49%'})

graph = html.Div([
       dcc.Store(id='selected_column'), html.Span(id='fig')
    ], style={'width': '49%', 'float': 'right'})

app.layout=html.Div([tabs,
                     graph,
                     ])

@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab')
)
def switch_tab(column_type):
    return generate_table(column_type)

@app.callback(
    Output('selected_column', 'data'),
    Input({"type": "button", "id": ALL}, "n_clicks"),
)
def get_triggered_column(clicks):
    if not any(clicks):
        return None
    
    triggered_id = ctx.triggered[0]["prop_id"].split(".")[0]
    triggered_column = json.loads(triggered_id)['id']

    return triggered_column
        
@app.callback(
    Output('fig', 'children'),
    Input('selected_column', 'data'),
    Input('tabs', 'active_tab'),
)
def set_display_children(selected_column, selected_column_type):
    if selected_column == None:
        return 'please select a column'
    
    return f'you choosed {selected_column} from {selected_column_type} column'

app.run(debug=True)